In [1]:
import numpy as np
from bezier_interspace_transforms import tendon_disp_to_bezier_control_points
# from scipy.optimize import fsolve

In [2]:
n_mid_points = 1
a = np.linspace(0, 1, n_mid_points + 2)[1:]
print(a)

[0.5 1. ]


In [3]:
# 定义点
A = np.array([1, 1, 0])
B = np.array([0, 0, 0])
C = np.array([2, 0, 0])

# 计算向量
vec_AB = B - A
vec_AC = C - A

# 平面法向量
normal_ABC = np.cross(vec_AB, vec_AC)

# 计算中点
M_AB = (A + B) / 2
M_AC = (A + C) / 2

# 中垂线方向向量
direction_AB = np.cross(normal_ABC, vec_AB)
direction_AC = np.cross(normal_ABC, vec_AC)

print(direction_AB, direction_AC)

# 设定两条中垂线的方程
# 使用矩阵方法解交点
# 方程: M_AB + t * direction_AB = M_AC + s * direction_AC
# 转换为线性方程组求解 t 和 s
# A = np.column_stack((direction_AB, -direction_AC))
# b = M_AC - M_AB

# try:
#     t_s = np.linalg.solve(A, b)
#     circle_center = M_AB + t_s[0] * direction_AB
#     print("圆心 (Circle Center):", circle_center)
# except np.linalg.LinAlgError:
#     print("中垂线可能不相交或存在多个解")

# 解 t
try:
    t = (M_AC[0] - M_AB[0]) / (direction_AB[0] + direction_AC[0])
    circle_center = M_AB + t * direction_AB
    # 计算半径
    radius = np.linalg.norm(circle_center - A)
    print("圆心 (Circle Center):", circle_center)
    print("半径 (Radius):", radius)
except ZeroDivisionError:
    print("方向向量x分量之和为0，无法解出t")

[ 2 -2  0] [2 2 0]
圆心 (Circle Center): [1. 0. 0.]
半径 (Radius): 1.0


In [4]:
def calculate_radius(A, B, C):
    # 计算向量
    vec_AB = B - A
    vec_AC = C - A

    # 平面法向量
    normal_ABC = np.cross(vec_AB, vec_AC)

    # 计算中点
    M_AB = (A + B) / 2
    M_AC = (A + C) / 2

    # 中垂线方向向量
    direction_AB = np.cross(normal_ABC, vec_AB)
    direction_AC = np.cross(normal_ABC, vec_AC)

    t = (M_AC[0] - M_AB[0]) / (direction_AB[0] + direction_AC[0])
    circle_center = M_AB + t * direction_AB
    # 计算半径
    radius = np.linalg.norm(circle_center - A)
    
    return radius, circle_center, normal_ABC

In [5]:
A = np.array([-2, 0, 3])
B = np.array([-4, -2, 3])
C = np.array([0, -2, 3])

radius, circle_center, _ = calculate_radius(A, B, C)

print(radius, circle_center)

2.0 [-2. -2.  3.]


In [6]:
# def angle_between_vectors(n1, n2):
#     # 计算两个向量的点积
#     dot_product = np.dot(n1, n2)
#     # 计算两个向量的模
#     norm_n1 = np.linalg.norm(n1)
#     norm_n2 = np.linalg.norm(n2)
#     # 计算余弦值
#     cos_theta = dot_product / (norm_n1 * norm_n2)
#     angle = np.arccos(cos_theta)
#     return angle

In [7]:
def angle_between_vectors(A, B):
    # 计算两个向量的点积
    dot_product = np.dot(A, B)
    # 计算两个向量的模
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    # 计算余弦值
    cos_theta = dot_product / (norm_A * norm_B)
    # cos_theta = np.clip(dot_product / (norm_A * norm_B), -1.0, 1.0)
    # 计算夹角的绝对值（弧度）
    angle = np.arccos(cos_theta)
    
    # 计算向量的叉积
    cross_product = np.cross(A, B)
    # 判断旋转方向
    if cross_product[2] < 0:
        # 顺时针旋转，角度取负
        angle = -angle
    
    return angle

In [8]:
def bezier_control_points_to_tendon_disp(p_0, p_1, p_2, l, r):
    """
    Convert from Bezier curve model model to constant curvature using optimization.
    """
    p_start = p_0[:-1]
    p_mid = p_1[:-1]
    p_end = p_2[:-1]
    
    s_bezier = 0.5
    # bezier_mid = p_start * (s_bezier - 1) ** 2 + p_mid * 4 * s_bezier * (1 - s_bezier) + p_end * s_bezier * (3 * s_bezier - 2)
    bezier_mid = 0.5 * (p_mid + 0.5*p_start + 0.5*p_end)
    radius, _, n_circle = calculate_radius(bezier_mid, p_start, p_end)
    k = 1 / radius
    delta = k * l * r
    n_base = np.array([0, -1, 0])
    phi = angle_between_vectors(n_base, n_circle)
    
    delta_x_solution = delta * np.cos(phi)
    delta_y_solution = delta * np.sin(phi)
        
    return delta_x_solution, delta_y_solution

In [9]:
p_0_h = np.array([2e-2, 2e-3, 0, 1])
l = 0.2
r = 0.01
ux = -0.003
uy = 0.001
# ux = -0.0753
# uy = 0.1626

In [10]:
p1, p2 = tendon_disp_to_bezier_control_points(ux, uy, l, r, p_0_h)
ux_solve, uy_solve = bezier_control_points_to_tendon_disp(p_0_h, p1, p2, l, r)

print(ux_solve, uy_solve)

ux_error = abs((ux_solve - ux) / ux) * 100 if ux != 0 else 0
uy_error = abs((uy_solve - uy) / uy) * 100 if uy != 0 else 0
print(f"ux: {ux:.4f}, uy: {uy:.4f} -> ux_solve: {ux_solve:.4f}, uy_solve: {uy_solve:.4f}, ux_error: {ux_error:.2f}%, uy_error: {uy_error:.2f}%")

-0.0029129736332490623 0.0009709912110830195
ux: -0.0030, uy: 0.0010 -> ux_solve: -0.0029, uy_solve: 0.0010, ux_error: 2.90%, uy_error: 2.90%


In [11]:
# 随机生成ux和uy
np.random.seed(0)  # 固定随机数种子以便结果可复现
n_samples = 1000
ux_values = np.random.uniform(-0.005, 0.005, n_samples)
uy_values = np.random.uniform(-0.005, 0.009, n_samples)

errors = []

for ux, uy in zip(ux_values, uy_values):
    p1, p2 = tendon_disp_to_bezier_control_points(ux, uy, l, r, p_0_h)
    ux_solve, uy_solve = bezier_control_points_to_tendon_disp(p_0_h, p1, p2, l, r)
    
    ux_error = abs((ux_solve - ux) / ux) * 100 if ux != 0 else 0
    uy_error = abs((uy_solve - uy) / uy) * 100 if uy != 0 else 0
    
    errors.append((ux, uy, ux_solve, uy_solve, ux_error, uy_error))
    
# 计算平均误差和统计误差超过50%的样本百分比
total_ux_error = sum(error[4] for error in errors)
total_uy_error = sum(error[5] for error in errors)
average_ux_error = total_ux_error / n_samples
average_uy_error = total_uy_error / n_samples

large_errors_count = sum(1 for error in errors if error[4] > 50 or error[5] > 50)
large_errors_percentage = (large_errors_count / n_samples) * 100

# # 打印结果
# for ux, uy, ux_solve, uy_solve, ux_error, uy_error in errors:
#     print(f"ux: {ux:.4f}, uy: {uy:.4f} -> ux_solve: {ux_solve:.4f}, uy_solve: {uy_solve:.4f}, ux_error: {ux_error:.2f}%, uy_error: {uy_error:.2f}%")

In [12]:
print(f"\n平均 ux 误差: {average_ux_error:.2f}%")
print(f"平均 uy 误差: {average_uy_error:.2f}%")
print(f"误差超过 50% 的样本百分比: {large_errors_percentage:.2f}%")

# 打印所有误差超过 50% 的例子
print("\n误差超过 50% 的例子:")
for ux, uy, ux_solve, uy_solve, ux_error, uy_error in errors:
    if ux_error > 50 or uy_error > 50:
        print(f"ux: {ux:.4f}, uy: {uy:.4f} -> ux_solve: {ux_solve:.4f}, uy_solve: {uy_solve:.4f}, ux_error: {ux_error:.2f}%, uy_error: {uy_error:.2f}%")


平均 ux 误差: 2.52%
平均 uy 误差: 2.52%
误差超过 50% 的样本百分比: 0.00%

误差超过 50% 的例子:


In [13]:
# 过滤误差超过100%的数据
filtered_errors = [error for error in errors if error[4] <= 100 and error[5] <= 100]

# 计算过滤后的平均误差
if filtered_errors:
    total_filtered_ux_error = sum(error[4] for error in filtered_errors)
    total_filtered_uy_error = sum(error[5] for error in filtered_errors)
    average_filtered_ux_error = total_filtered_ux_error / len(filtered_errors)
    average_filtered_uy_error = total_filtered_uy_error / len(filtered_errors)
else:
    average_filtered_ux_error = 0
    average_filtered_uy_error = 0

print(f"\n去掉超过100%误差的数据后，平均 ux 误差: {average_filtered_ux_error:.2f}%")
print(f"去掉超过100%误差的数据后，平均 uy 误差: {average_filtered_uy_error:.2f}%")



去掉超过100%误差的数据后，平均 ux 误差: 2.52%
去掉超过100%误差的数据后，平均 uy 误差: 2.52%


In [14]:
# def equations(vars, p1_target, p2_target, l, r, p0):
#     delta_x, delta_y = vars
#     p1, p2 = tendon_disp_to_bezier_control_points(delta_x, delta_y, l, r, p0)
#     eq1 = p1 - p1_target
#     eq2 = p2 - p2_target
#     return np.hstack((eq1, eq2))

# def bezier_control_points_to_tendon_disp_2(p1_target, p2_target, l, r, p0):
#     initial_guess = [0.003, 0.003]
#     result = fsolve(equations, initial_guess, args=(p1_target, p2_target, l, r, p0))
#     return result


In [15]:
# p_0_h = np.array([2e-2, 2e-3, 0, 1])
# l = 0.2
# r = 0.01
# ux = 0.005
# uy = 0.003

# p1, p2 = tendon_disp_to_bezier_control_points(ux, uy, l, r, p_0_h)
# ux_solve, uy_solve = bezier_control_points_to_tendon_disp_2(p_0_h, p1, p2, l, r)

# print(ux_solve, uy_solve)